# Projet final du Kit Big Data 2022

## Importation des librairies à utilisées

In [28]:
import pandas as pd
import re
import os
import shutil
import uuid
import xlwings as xw
import numpy as np

import time
import datetime
from unidecode import unidecode

import requests
from bs4 import BeautifulSoup
from urllib import request
from shutil import copyfileobj


## 1. Acquisition et chargement des données 

### Acquisition de l'ensemble des fichiers Excel des classements et mise en place de leur copie locale

In [10]:
# extract list of rankings date

def List_ranking_date(url="https://www.vendeeglobe.org/fr/classement"):
    
    req = requests.get(url) # make request

    soup = BeautifulSoup(req.content) # extract content

    soup_info = soup.find('select', class_='form__input m--select onsubmit-rank') # extract specific information

    ranking_date = soup_info.find_all('option')
    list_ranking_date = [date_ranking.get('value') for date_ranking in ranking_date[1:]]
    list_ranking_date = list(set(list_ranking_date)) # Remove duplicates
    
    return list_ranking_date

list_ranking_date = List_ranking_date()
print(f"{len(list_ranking_date)} rankings dates extracted")
print(list_ranking_date[0:5])

703 rankings dates extracted
['20201114_140000', '20210129_040000', '20210108_080000', '20201214_110000', '20210219_170000']


In [11]:
### Download Excel rankings data

# Define a function to download xlsx file
def Excel_rankings_data(ranking_date, excel_data_dir='', domain='https://www.vendeeglobe.org', page='/download-race-data/vendeeglobe_{date}.xlsx'):
    
    for i, ranking_date in enumerate(ranking_date):
        
        url = domain + page.format(date=ranking_date) # define url
        file = excel_data_dir + os.path.basename(page.format(date=ranking_date)) # define a destination file name and a path
                
        resp = request.urlopen(url) # get the response file object from url
        dest_file = open(file, 'wb') # Open the destination file in write mode and get the file object
        copyfileobj(resp, dest_file)  # copy the contents of response file object to destination file object
        
        #with request.urlopen(url) as response, open(filename, 'wb') as out_file:
        #copyfileobj(response, out_file)


# Create folder if does not exist 
excel_ranking_data_dir = 'data/' + 'ranking_data/'
if not os.path.exists(excel_ranking_data_dir):
        os.makedirs(excel_ranking_data_dir)

# Apply function to download Excel rankings data in directory

Excel_rankings_data(list_ranking_date, excel_data_dir=excel_ranking_data_dir)
print("Excel files upload completed.")


Excel files upload completed.


### Chargement des données de classement juste avant les arrivées des voiliers

In [33]:
# reading/writing Excel file with xlwings
def save_with_xlwings(file, directory):
    tempfile = directory + '{uuid.uuid1()}.xlsx'
    excel_app = xw.App(visible=False)
    excel_book = excel_app.books.open(file)
    excel_book.save(tempfile)
    excel_book.close()
    excel_app.quit()
    return tempfile


# Function to find out if the skipper is still in the race (not arrived) or if he arrived
def skipper_status_in_race(rank):
    
    if (isinstance(rank, str)):
        if rank.strip().isnumeric():
            skipper_status = 'not arrived'   
        else:
            if 'ARV' in rank:
                skipper_status = 'arrived'
            elif 'RET' in rank:
                skipper_status = 'not arrived'
            else:
                skipper_status = np.nan            
    else:
        skipper_status = np.nan
    return skipper_status


# Function to extract rankings date and time
def ranking_date_time(data):
    
    # change name format of month
    modif_month = dict(janvier=1, fevrier=2, mars=3, avril=4, mai=5, juin=6, juillet=7, aout=8, septembre=9, octobre=10, novembre=11, decembre=12)
    
    date_time = ' '.join(data.iloc[1][1].split()[3:-1]) # extract datetime

    # separate datetime into day, month, year, time
    date_time = re.sub(' à ', ' ', date_time)
    day, month, year, fr_time = tuple(date_time.split())

    # Convert time from 15h00 to 03:00PM)
    fr_hour, fr_min = tuple(fr_time.split('h'))
    time = datetime.time(int(fr_hour), int(fr_min))

    # Convert date from 27 fevrier 2021 to 2021-02-27)
    date = datetime.datetime(int(year), modif_month[unidecode(month).lower()], int(day))
    
    date_time = datetime.datetime.combine(date, time) # obtain rankings date and time using date and time
        
    return date_time


# Function to load rankings data (from xslx files to dataframes)just before the arrival of the boats
def rankings_data(directory):
    
    rankings_data_skippers_not_arrived = pd.DataFrame()
    rankings_data_skippers_arrived = pd.DataFrame()

    xslx_files_paths = [directory+file for file in os.listdir(directory)]
    for i, file in enumerate(xslx_files_paths):
                
        # Read excel file using xlwings
        #uuid_dir = 'data/' + 'uuid/' # uuid folder
        #if not os.path.exists(uuid_dir):
         #   os.makedirs(uuid_dir)     # Create folder if not exit
        #file_uuid = save_with_xlwings(file, uuid_dir)
        rankings_data_skippers = pd.read_excel(file) #file_uuid

        # Insert ranking date
        rankings_data_skippers.insert(0, 'date_time', ranking_date_time(rankings_data_skippers))
        
        # Map skippers status (skippers arrived (finished the race), skippers not arrived (still in race or abandonned), NaN: other)
        rankings_data_skippers = rankings_data_skippers.rename(columns={'Unnamed: 0':'skipper_status'})
        rankings_data_skippers['skipper_status'] = rankings_data_skippers[list(rankings_data_skippers.columns)[2]].apply(lambda x: skipper_status_in_race(x))

        # Add skippers to dfs
        rankings_data_skippers_arrived = pd.concat([rankings_data_skippers_arrived, rankings_data_skippers[rankings_data_skippers['skipper_status'] == 'arrived']], ignore_index=True)
        rankings_data_skippers_not_arrived = pd.concat([rankings_data_skippers_not_arrived, rankings_data_skippers[rankings_data_skippers['skipper_status'] == 'not arrived']], ignore_index=True)
    
    return rankings_data_skippers_not_arrived, rankings_data_skippers_arrived


rankings_data_skippers_not_arrived, rankings_data_skippers_arrived = rankings_data(excel_ranking_data_dir)


In [36]:
rankings_data_skippers_not_arrived

,date_time,skipper_status,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,2021-01-29 18:00:00,not arrived,10,\nFRA 53,Maxime Sorel\nV And B Mayenne,17:30 FR\n,46°07.29'N,06°13.58'W,82°,19.1 kts,...,72°,15.1 kts,15.0 kts,45.3 nm,49°,12.4 kts,12.3 kts,298.4 nm,185.1 nm,0.0 nm
1,2021-01-29 18:00:00,not arrived,11,FR\nFRA 02,Armel Tripon\nL'Occitane en Provence,17:30 FR\n,41°40.31'N,10°38.49'W,5°,8.5 kts,...,17°,7.5 kts,7.3 kts,22.4 nm,62°,7.3 kts,7.1 kts,175.1 nm,482.6 nm,297.5 nm
2,2021-01-29 18:00:00,not arrived,12,\nFRA 30,Clarisse Cremer\nBanque Populaire X,17:30 FR\n,36°26.28'N,29°36.29'W,353°,8.1 kts,...,9°,8.7 kts,6.1 kts,26.2 nm,10°,11.9 kts,9.3 kts,285.5 nm,1379.1 nm,1194.0 nm
3,2021-01-29 18:00:00,not arrived,13,\nFRA 49,Romain Attanasio\nPure - Best Western Hotels a...,17:30 FR\n,23°59.68'N,34°28.19'W,14°,10.4 kts,...,18°,10.8 kts,10.0 kts,32.5 nm,11°,11.3 kts,10.2 kts,271.9 nm,2069.2 nm,1884.1 nm
4,2021-01-29 18:00:00,not arrived,14,\nFRA 8,Jérémie Beyou\nCharal,17:30 FR\n,22°51.11'N,36°30.29'W,9°,16.6 kts,...,2°,15.2 kts,12.0 kts,45.6 nm,356°,16.7 kts,12.9 kts,400.0 nm,2194.6 nm,2009.5 nm
5,2021-01-29 18:00:00,not arrived,15,\nFRA 14,Arnaud Boissieres\nLa Mie Câline - Artisans Ar...,17:30 FR\n,10°51.53'N,35°18.82'W,334°,15.8 kts,...,339°,15.6 kts,9.8 kts,46.8 nm,337°,12.5 kts,8.2 kts,300.3 nm,2741.5 nm,2556.4 nm
6,2021-01-29 18:00:00,not arrived,16,\nSUI 7,Alan Roura\nLa Fabrique,17:30 FR\n,07°39.50'N,34°49.21'W,340°,12.7 kts,...,336°,13.2 kts,8.4 kts,39.5 nm,330°,11.8 kts,7.2 kts,282.7 nm,2895.6 nm,2710.4 nm
7,2021-01-29 18:00:00,not arrived,17,\nJPN 11,Kojiro Shiraishi\nDMG MORI Global One,17:30 FR\n,07°11.91'N,34°29.06'W,331°,15.1 kts,...,335°,16.0 kts,10.0 kts,48.1 nm,330°,12.0 kts,7.5 kts,288.6 nm,2911.1 nm,2725.9 nm
8,2021-01-29 18:00:00,not arrived,18,\nFRA 92,Stéphane Le Diraison\nTime For Oceans,17:30 FR\n,06°30.68'N,34°23.16'W,337°,12.6 kts,...,343°,13.4 kts,9.9 kts,40.3 nm,339°,10.1 kts,7.4 kts,241.5 nm,2945.4 nm,2760.2 nm
9,2021-01-29 18:00:00,not arrived,19,\nGBR 777,Pip Hare\nMedallia,17:30 FR\n,04°51.03'N,34°32.62'W,336°,11.1 kts,...,341°,11.0 kts,7.9 kts,32.9 nm,343°,7.6 kts,5.8 kts,182.3 nm,3039.3 nm,2854.1 nm


In [35]:
rankings_data_skippers_arrived

,date_time,skipper_status,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,2021-01-29 18:00:00,arrived,1\nARV,\nFRA 17,Yannick Bestaven\nMaître Coq IV,NaN,NaN,NaN,28/01/2021 04:19:46 FR,80j 03h 44min 46s\n-10h 15min 00s,...,NaN,NaN,NaN,NaN,NaN,12.6 kts,24365.7 nm,117.3 %,14.8 kts,28583.8 nm
1,2021-01-29 18:00:00,arrived,2\nARV,\nFRA 79,Charlie Dalin\nAPIVIA,NaN,NaN,NaN,27/01/2021 20:35:47 FR,80j 06h 15min 47s\n,...,NaN,NaN,02h 31min 01s,NaN,02h 31min 01s,12.6 kts,24365.7 nm,119.6 %,15.1 kts,29135.0 nm
2,2021-01-29 18:00:00,arrived,3\nARV,\nFRA 18,Louis Burton\nBureau Vallée 2,NaN,NaN,NaN,28/01/2021 00:45:12 FR,80j 10h 25min 12s\n,...,NaN,NaN,06h 40min 26s,NaN,04h 09min 25s,12.6 kts,24365.7 nm,117.6 %,14.8 kts,28650.0 nm
3,2021-01-29 18:00:00,arrived,4\nARV,\nFRA 01,Jean Le Cam\nYes we Cam !,NaN,NaN,NaN,28/01/2021 20:19:55 FR,80j 13h 44min 55s\n-16h 15min 00s,...,NaN,NaN,10h 00min 09s,NaN,03h 19min 43s,12.5 kts,24365.7 nm,112.9 %,14.1 kts,27501.5 nm
4,2021-01-29 18:00:00,arrived,5\nARV,\nMON 10,Boris Herrmann\nSeaexplorer - Yacht Club De Mo...,NaN,NaN,NaN,28/01/2021 11:19:45 FR,80j 14h 59min 45s\n-06h 00min 00s,...,NaN,NaN,11h 14min 59s,NaN,01h 14min 50s,12.6 kts,24365.7 nm,116.8 %,14.7 kts,28448.5 nm
5,2021-01-29 18:00:00,arrived,6\nARV,\nFRA 59,Thomas Ruyant\nLinkedOut,NaN,NaN,NaN,28/01/2021 05:42:01 FR,80j 15h 22min 01s\n,...,NaN,NaN,11h 37min 15s,NaN,22min 16s,12.6 kts,24365.7 nm,119.7 %,15.1 kts,29175.5 nm
6,2021-01-29 18:00:00,arrived,7\nARV,\nFRA 1000,Damien Seguin\nGroupe APICIL,NaN,NaN,NaN,28/01/2021 12:18:20 FR,80j 21h 58min 20s\n,...,NaN,NaN,18h 13min 34s,NaN,06h 36min 19s,12.5 kts,24365.7 nm,112.9 %,14.2 kts,27512.3 nm
7,2021-01-29 18:00:00,arrived,8\nARV,\nITA 34,Giancarlo Pedote\nPrysmian Group,NaN,NaN,NaN,28/01/2021 13:02:20 FR,80j 22h 42min 20s\n,...,NaN,NaN,18h 57min 34s,NaN,44min 00s,12.5 kts,24365.7 nm,116.9 %,14.7 kts,28489.9 nm
8,2021-01-29 18:00:00,arrived,9\nARV,\nFRA 09,Benjamin Dutreux\nOMIA - Water Family,NaN,NaN,NaN,29/01/2021 10:05:20 FR,81j 19h 45min 20s\n,...,NaN,NaN,1j 16h 00min 34s,NaN,21h 03min 00s,12.4 kts,24365.7 nm,114.2 %,14.2 kts,27832.5 nm
9,2021-02-02 18:00:00,arrived,1\nARV,\nFRA 17,Yannick Bestaven\nMaître Coq IV,NaN,NaN,NaN,28/01/2021 04:19:46 FR,80j 03h 44min 46s\n-10h 15min 00s,...,NaN,NaN,NaN,NaN,NaN,12.6 kts,24365.7 nm,117.3 %,14.8 kts,28583.8 nm


### Acquisition des caractéristiques des bateaux

In [37]:
### extract boats characteristics

# Define a function to extract boats characteristics
def Boats_characteristics(url="https://www.vendeeglobe.org/fr/glossaire"):
    
    List_boats_charact = []

    req = requests.get(url) # make request

    soup = BeautifulSoup(req.content) # extract all information

    # extract specific information
    boats_infos = soup.find_all('div', class_='boats-list__infos')
    boats_name = [info.find('h3').text.strip() for info in boats_infos]
    skippers_name = [info.find('span').text.strip() for info in boats_infos]
    
    boats_pop_infos = soup.find_all('div', class_='boats-list__popup-infos')
    for i, info in enumerate(boats_pop_infos):
        pop_boats_infos = info.find_all('li')

        # Retrieve the boat characteristics
        characteristics = [tuple(characteristics.text.split(' : ', maxsplit=1)) for characteristics in pop_boats_infos]
        characteristics = dict(map(reversed, characteristics))
        characteristics = {j: k for k, j in characteristics.items()}
        other_characteristics = dict(XNom_skipper=skippers_name[i], XNom_bateau=boats_name[i])
        other_characteristics.update(characteristics)
        List_boats_charact.append(other_characteristics)

    boats_characteristics = pd.DataFrame(List_boats_charact)
    return boats_characteristics


# Apply function to extract boats characteristics data
boats_characteristics = Boats_characteristics()

# Save extract data to csv file in local folder
boat_data_dir = 'data/' + 'boat_data/' # folder name
if not os.path.exists(boat_data_dir):
        os.makedirs(boat_data_dir)     # Create folder if not exit
file = boat_data_dir + 'boats_characteristics.csv' # define a destination file name and a path
boats_characteristics.to_csv(file, index=False)  
#boats_characteristics = pd.read_csv(file)

boats_characteristics


,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Déplacement (poids),Hauteur mât,Largeur,Longueur,Nombre de dérives,Numéro de voile,Surface de voiles au portant,Surface de voiles au près,Tirant d'eau,Voile quille,XNom_bateau,XNom_skipper
0,"No Way Back, Vento di Sardegna",VPLP/Verdier,Persico Marine,01 Août 2015,7 t,29 m,"5,85 m","18,28 m",foils,FRA 56,570 m2,320 m2,"4,50 m",monotype,NEWREST - ART & FENÊTRES,Fabrice AMEDEO
1,"Gitana Eighty, Synerciel, Newrest-Matmut",Bruce Farr Design,Southern Ocean Marine (Nouvelle Zélande),08 Mars 2007,9t,28m,"5,80m","18,28m",2,FRA 49,560 m2,280 m2,"4,50m",acier forgé,PURE - Best Western®,Romain ATTANASIO
2,"Famille Mary-Etamine du Lys, Initiatives Coeur...",Marc Lombard,MAG France,01 Mars 1998,9t,29 m,"5,54m","18,28m",2,FRA72,580 m2,260 m2,"4,50m",acier,TSE - 4MYPLANET,Alexia BARRIER
3,Safran 2 - Des Voiles et Vous,Verdier - VPLP,CDK Technologies,12 Mars 2015,8 t,29 m,"5,80 m","18,28 m",foils,17,550 m2,310 m2,"4,50 m",acier mécano soudé,Maître CoQ IV,Yannick BESTAVEN
4,NaN,VPLP,CDK Technologies,18 Août 2018,8t,29 m,"5,85 m","18,28 m",foils,08,600 m2,320 m2,"4,50 m",acier,CHARAL,Jérémie BEYOU
5,"Ecover3, Président, Gamesa, Kilcullen Voyager-...",Owen Clarke Design LLP - Clay Oliver,Hakes Marine - Mer Agitée,03 Août 2007,"7,9 tonnes",29 m,"5,65 m","18,28 m",foils,FRA 14,610 m²,300 m²,"4,50 m",basculante avec vérin,LA MIE CÂLINE - ARTISANS ARTIPÔLE,Arnaud BOISSIÈRES
6,Banque Populaire VIII,Verdier - VPLP,CDK Technologies,09 Juin 2015,"7,6 t",28 m,"5,80 m","18,28 m",foils,18,600 m2,300 m2,"4,50 m",acier,BUREAU VALLEE 2,Louis BURTON
7,Kingfisher - Educacion sin Fronteras - Forum M...,Owen Clarke Design,Martens Yachts,02 Février 2000,"8,9 t",26 m,"5,30 m","18,28 m",2,ESP 33,470 m2,240 m2,"4,50 m",acier,ONE PLANET ONE OCEAN,Didac COSTA
8,"Paprec-Virbac2, Estrella Damm, We are Water, L...",Bruce Farr Yacht Design,Southern Ocean Marine (Nouvelle-Zélande),02 Février 2007,9 t,"28,50","5,80 m","18,28 m",2 asymétriques,FRA 71,560 m2,270 m2,"4,50 m",basculante sur vérin hydraulique,GROUPE SÉTIN,Manuel COUSIN
9,Macif - SMA,Verdier - VPLP,CDK - Mer Agitée,01 Mars 2011,"7,7 t",29 m,"5,70 m","18,28 m",2,FRA30,570 m2,340 m2,"4,5 m",acier forgé,BANQUE POPULAIRE X,Clarisse CREMER


## 2. Préparation des données

### Préparation des données relatives aux classements

In [46]:
# preprocessing of data

df = rankings_data_skippers_not_arrived.copy()

df.rename(columns = {'Unnamed: 1':'Rang','Unnamed: 2':'Voile','Unnamed: 3':'Bateau',\
                            'Unnamed: 4':'Heure FR','Unnamed: 5':'Latitude','Unnamed: 6':'Longitude',\
                            'Unnamed: 7':'Cap30m (°)','Unnamed: 8':'Vitesse30m (kts)','Unnamed: 9':'VMG30m (kts)',\
                            'Unnamed: 10':'Distance30m (mètre)','Unnamed: 11':'Cap (°)','Unnamed: 12':'Vitesse (kts)',\
                            'Unnamed: 13':'VMG (kts)','Unnamed: 14':'Distance (mètre)','Unnamed: 15':'Cap24h (°)',\
                            'Unnamed: 16':'Vitesse24h (kts)','Unnamed: 17':'VMG24h (kts)','Unnamed: 18':'Distance24h (mètre)',\
                            'Unnamed: 19':'DFT (nm)','Unnamed: 20':'DTL (nm)'}, inplace = True)
    
df = df.drop(columns = ["skipper_status"])
    
df['Rang'] = df['Rang'].apply(lambda x: '-1' if x == 'RET' else x)
    
df['Voile'] = df['Voile'].fillna("")
nat = df['Voile'].apply(lambda x: x.split(' ')[0])
nat = nat.apply(lambda x: x.split('\n')[1])
ch = df['Voile'].apply(lambda x: x.split(' ')[1])
df.insert(2, 'Nationalité', nat)
df.insert(3, 'Numéro du bateau', ch)
df = df.drop(columns = ["Voile"])
    
df['Bateau'] = df['Bateau'].fillna("")
names = list(df['Bateau']) 
prenom, nom, architecte = [],[], []
for i, name in enumerate(names):
    temp, temp_archi = name.split('\n')
    architecte.append(temp_archi)
    if len(temp.split())==2:
        prenom.append(temp.split()[0])
        nom.append(temp.split()[1])
    else:
        prenom.append(temp.split()[0])
        a,b = temp.split()[1:3]
        nom.append(a +' '+b)
df.insert(2, 'Nom', nom)
df['Nom'] = df['Nom'].str.upper()
df.insert(3, 'Prenom', prenom)
df.insert(6, 'Architecte', architecte)
df = df.drop(columns = ["Bateau"])
    

liste_cap =['Cap30m (°)', 'Cap (°)', 'Cap24h (°)']
for j,col in enumerate(liste_cap):
    df[col] = df[col].fillna('')
    df[col] =df[col].apply(lambda x: str(x)[:-1])   
liste_vitesse = ['Vitesse30m (kts)', 'VMG30m (kts)','Vitesse (kts)', 'VMG (kts)','Vitesse24h (kts)', 'VMG24h (kts)']
for j,col in enumerate(liste_vitesse):
    df[col] = df[col].fillna('')
    df[col] =df[col].apply(lambda x: str(x)[:-3])  
liste_nm = ['Distance30m (mètre)','Distance (mètre)','Distance24h (mètre)','DFT (nm)','DTL (nm)']
for j,col in enumerate(liste_nm):
    df[col] = df[col].fillna('')
    df[col] =df[col].apply(lambda x: str(x)[:-2])  

df['Heure FR'] = df['Heure FR'].apply(lambda x: datetime.datetime.strptime(x.split(' FR')[0], '%H:%M').strftime('%H:%M:%S') if isinstance(x, str) else np.nan)


process_rankings_data_skippers_not_arrived = df.copy()

process_rankings_data_skippers_not_arrived.head()


,date_time,Rang,Nom,Prenom,Nationalité,Numéro du bateau,Architecte,Heure FR,Latitude,Longitude,...,Cap (°),Vitesse (kts),VMG (kts),Distance (mètre),Cap24h (°),Vitesse24h (kts),VMG24h (kts),Distance24h (mètre),DFT (nm),DTL (nm)
0,2021-01-29 18:00:00,10,SOREL,Maxime,FRA,53,V And B Mayenne,17:30:00,46°07.29'N,06°13.58'W,...,72,15.1,15.0,45.3,49,12.4,12.3,298.4,185.1,0.0
1,2021-01-29 18:00:00,11,TRIPON,Armel,FRA,02,L'Occitane en Provence,17:30:00,41°40.31'N,10°38.49'W,...,17,7.5,7.3,22.4,62,7.3,7.1,175.1,482.6,297.5
2,2021-01-29 18:00:00,12,CREMER,Clarisse,FRA,30,Banque Populaire X,17:30:00,36°26.28'N,29°36.29'W,...,9,8.7,6.1,26.2,10,11.9,9.3,285.5,1379.1,1194.0
3,2021-01-29 18:00:00,13,ATTANASIO,Romain,FRA,49,Pure - Best Western Hotels and Resorts,17:30:00,23°59.68'N,34°28.19'W,...,18,10.8,10.0,32.5,11,11.3,10.2,271.9,2069.2,1884.1
4,2021-01-29 18:00:00,14,BEYOU,Jérémie,FRA,8,Charal,17:30:00,22°51.11'N,36°30.29'W,...,2,15.2,12.0,45.6,356,16.7,12.9,400.0,2194.6,2009.5


### Extraction des caractéristiques techniques de chacun des voiliers

In [56]:
# change name format of month
def change_format_date(date):
    modif_month = dict(janvier=1, fevrier=2, mars=3, avril=4, mai=5, juin=6, juillet=7, aout=8, septembre=9, octobre=10, novembre=11, decembre=12)
    day, month, year = tuple(date.split())
    month = modif_month[unidecode(month).lower()]
    return datetime.datetime(int(year), int(month), int(day))

df = boats_characteristics.copy()

# # preprocessing of data

df['Nom bateau'] = df['XNom_bateau']
df['Date de lancement'] = df['Date de lancement'].apply(lambda x: change_format_date(x))

#liste = ['Longueur', 'Largeur', "Tirant d'eau",'Déplacement (poids)', 'Hauteur mât', 'Surface de voiles au près', 'Surface de voiles au portant']
#
df['Longueur'] = df['Longueur'].apply(lambda x: float(re.sub(r"([^0-9,]+)",'', x).replace(',', '.')) if (isinstance(x, str) and any([l.isdigit() for l in x])) else np.nan)
df['Largeur'] = df['Largeur'].apply(lambda x: float(re.sub(r"([^0-9,]+)",'', x).replace(',', '.')) if (isinstance(x, str) and any([l.isdigit() for l in x])) else np.nan)
df["Tirant d'eau"] = df["Tirant d'eau"].apply(lambda x: float(re.sub(r"([^0-9,]+)",'', x).replace(',', '.')) if (isinstance(x, str) and any([l.isdigit() for l in x])) else np.nan)
df['Déplacement (poids)'] = df['Déplacement (poids)'].apply(lambda x: float(re.sub(r"([^0-9,]+)",'', x).replace(',', '.')) if (isinstance(x, str) and any([l.isdigit() for l in x])) else np.nan)
df['Hauteur mât'] = df['Hauteur mât'].apply(lambda x: float(re.sub(r"([^0-9,]+)",'', x).replace(',', '.')) if (isinstance(x, str) and any([l.isdigit() for l in x])) else np.nan)
df['Surface de voiles au près'] = df['Surface de voiles au près'].apply(lambda x: float(x[:3]) if (isinstance(x, str) and len(x)>3) else np.nan)
df['Surface de voiles au portant'] = df['Surface de voiles au portant'].apply(lambda x: float(x[:3]) if (isinstance(x, str) and len(x)>3) else np.nan)

# 
drift_amount2no = {drift_amount:no for no, drift_amount in enumerate(df['Nombre de dérives'].dropna().unique().tolist())}
no2drift_amounts = {v:k for k,v in drift_amount2no.items()}
df['Nombre de dérives'] = df['Nombre de dérives'].apply(lambda x: drift_amount2no[x] if isinstance(x, str) else np.nan)

#
keel2no = {keel:no for no, keel in enumerate(df['Voile quille'].dropna().unique().tolist())}
no2keels = {v:k for k,v in keel2no.items()}
df['Voile quille'] = df['Voile quille'].apply(lambda x: keel2no[x] if isinstance(x, str) else np.nan)

# split variables
df['Prenom'] = df['XNom_skipper'].apply(lambda x: ''.join([word for word in x.split() if not word.isupper()]))
df['Nom'] = df['XNom_skipper'].apply(lambda x: unidecode(''.join([word for word in x.split() if word.isupper()])))
df['Numéro de voile'] = df['Numéro de voile'].apply(lambda x: ' '.join([x[:3], x[3:]]) if (isinstance(x, str) and (len(x) > 3)) else x)
df['Nationalité'] = df['Numéro de voile'].apply(lambda x: ''.join([l for l in x if (l.lower().isalpha())]) if isinstance(x, str) else np.nan)
df['Numéro du bateau'] = df['Numéro de voile'].apply(lambda x: ''.join([l for l in x if l.isdigit()]) if isinstance(x, str) else np.nan)
df = df.drop(columns=['XNom_bateau', 'XNom_skipper', 'Numéro de voile'])
df = df.reindex(columns=(['Nom', 'Prenom', 'Nationalité', 'Numéro du bateau', 'Nom bateau'] + list([col for col in df.columns if col not in ['Nom', 'Prenom', 'Nationalité', 'Numéro du bateau', 'Nom bateau']])))

process_boats_characteristics = df.copy()

process_boats_characteristics.head()


,Nom,Prenom,Nationalité,Numéro du bateau,Nom bateau,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Déplacement (poids),Hauteur mât,Largeur,Longueur,Nombre de dérives,Surface de voiles au portant,Surface de voiles au près,Tirant d'eau,Voile quille
0,AMEDEO,Fabrice,FRA,56,NEWREST - ART & FENÊTRES,"No Way Back, Vento di Sardegna",VPLP/Verdier,Persico Marine,2015-08-01,7.0,29.0,5.85,18.28,0.0,570.0,320.0,4.5,0.0
1,ATTANASIO,Romain,FRA,49,PURE - Best Western®,"Gitana Eighty, Synerciel, Newrest-Matmut",Bruce Farr Design,Southern Ocean Marine (Nouvelle Zélande),2007-03-08,9.0,28.0,5.80,18.28,1.0,560.0,280.0,4.5,1.0
2,BARRIER,Alexia,FRA,72,TSE - 4MYPLANET,"Famille Mary-Etamine du Lys, Initiatives Coeur...",Marc Lombard,MAG France,1998-03-01,9.0,29.0,5.54,18.28,1.0,580.0,260.0,4.5,2.0
3,BESTAVEN,Yannick,,17,Maître CoQ IV,Safran 2 - Des Voiles et Vous,Verdier - VPLP,CDK Technologies,2015-03-12,8.0,29.0,5.80,18.28,0.0,550.0,310.0,4.5,3.0
4,BEYOU,Jérémie,,08,CHARAL,NaN,VPLP,CDK Technologies,2018-08-18,8.0,29.0,5.85,18.28,0.0,600.0,320.0,4.5,2.0


### Rapprochement des données des voiliers avec celle des classements

In [58]:
# drop duplicates columns (in two dataframes) in one dataframe
boats_without_some_variables = process_boats_characteristics.drop(columns=['Prenom','Nationalité','Numéro du bateau', 'Architecte'])

# merge two dataframes
skippers_and_boats = pd.merge(process_rankings_data_skippers_not_arrived, boats_without_some_variables, on='Nom', how='inner')

skippers_and_boats.head()



,date_time,Rang,Nom,Prenom,Nationalité,Numéro du bateau,Architecte,Heure FR,Latitude,Longitude,...,Date de lancement,Déplacement (poids),Hauteur mât,Largeur,Longueur,Nombre de dérives,Surface de voiles au portant,Surface de voiles au près,Tirant d'eau,Voile quille
0,2021-01-29 18:00:00,10,SOREL,Maxime,FRA,53,V And B Mayenne,17:30:00,46°07.29'N,06°13.58'W,...,2007-09-07,7.7,29.0,5.5,18.28,1.0,700.0,365.0,4.5,2.0
1,2020-11-10 10:00:00,6,SOREL,Maxime,FRA,53,V And B Mayenne,09:30:00,43°14.58'N,09°17.24'W,...,2007-09-07,7.7,29.0,5.5,18.28,1.0,700.0,365.0,4.5,2.0
2,2020-12-05 22:00:00,11,SOREL,Maxime,FRA,53,V And B Mayenne,21:30:00,42°09.28'S,41°23.81'E,...,2007-09-07,7.7,29.0,5.5,18.28,1.0,700.0,365.0,4.5,2.0
3,2020-12-10 15:00:00,11,SOREL,Maxime,FRA,53,V And B Mayenne,14:30:00,42°32.05'S,76°33.98'E,...,2007-09-07,7.7,29.0,5.5,18.28,1.0,700.0,365.0,4.5,2.0
4,2020-11-21 05:00:00,14,SOREL,Maxime,FRA,53,V And B Mayenne,04:30:00,05°45.50'S,30°30.23'W,...,2007-09-07,7.7,29.0,5.5,18.28,1.0,700.0,365.0,4.5,2.0


In [63]:
skippers_and_boats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18169 entries, 0 to 18168
Data columns (total 37 columns):
date_time                       18169 non-null datetime64[ns]
Rang                            18169 non-null object
Nom                             18169 non-null object
Prenom                          18169 non-null object
Nationalité                     18169 non-null object
Numéro du bateau                18169 non-null object
Architecte                      18169 non-null object
Heure FR                        14211 non-null object
Latitude                        14211 non-null object
Longitude                       14211 non-null object
Cap30m (°)                      18169 non-null object
Vitesse30m (kts)                18169 non-null object
VMG30m (kts)                    18169 non-null object
Distance30m (mètre)             18169 non-null object
Cap (°)                         18169 non-null object
Vitesse (kts)                   18169 non-null object
VMG (kts)          